# Initializing with setup 

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')
# After executing, drive files will be present in "/content/drive/My Drive".

Mounted at /content/drive


In [ ]:
!pip install stanza

In [ ]:
!pip install -U spacy==2.1.0 

!python -m spacy download en
!python -m spacy download en_core_web_lg 

!pip uninstall -y neuralcoref 

!pip install neuralcoref --no-binary neuralcoref

In [ ]:
import stanza
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

import os
os.environ["CORENLP_HOME"] = corenlp_dir

2022-04-08 10:18:09 INFO: Installing CoreNLP package into ./corenlp...


2022-04-08 10:18:19 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


In [ ]:
# !ls $CORENLP_HOME

## Start stanza server [Only can run on Google colab]

In [ ]:
# Only can run on Google colab
# Import client module
from stanza.server import CoreNLPClient
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    # annotators=['tokenize', 'ner'], 
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner'], 
    # annotators=['ner'], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

2022-04-08 10:21:42 INFO: Writing properties to tmp file: corenlp_server-a4cf1673d5d3499e.props
2022-04-08 10:21:42 INFO: Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a4cf1673d5d3499e.props -annotators tokenize,ssplit,pos,lemma,ner -preload -outputFormat serialized


In [ ]:
!ps -o pid,cmd | grep java

# End of initializing

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import json
import unicodedata

import nltk
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')

# data_file = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/cnbc.csv')
cnbc = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/cnbc.csv')
nytimes = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/nytimes.csv')
bbc = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/bbc.csv')
ukrin = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/ukrinform.csv')
cna = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/cna.csv')
tass = pd.read_csv('/content/drive/My Drive/20212022T2/TM/dataset/tass.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
cnbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      190 non-null    object
 1   posted_at  190 non-null    object
 2   body       166 non-null    object
dtypes: object(3)
memory usage: 4.6+ KB


In [ ]:
tass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   484 non-null    object
 1   body    475 non-null    object
dtypes: object(2)
memory usage: 7.7+ KB


In [ ]:
nytimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      52 non-null     object
 1   posted_at  52 non-null     object
 2   body       51 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [ ]:
bbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538 entries, 0 to 537
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      538 non-null    object
 1   posted_at  538 non-null    object
 2   body       337 non-null    object
dtypes: object(3)
memory usage: 12.7+ KB


In [ ]:
ukrin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      30 non-null     object
 1   posted_at  30 non-null     object
 2   body       30 non-null     object
dtypes: object(3)
memory usage: 848.0+ bytes


In [ ]:
cna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      40 non-null     object
 1   posted_at  40 non-null     object
 2   body       40 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [ ]:
# Remove missing value
cnbc.dropna(inplace = True)
nytimes.dropna(inplace = True)
bbc.dropna(inplace = True)
ukrin.dropna(inplace = True)
cna.dropna(inplace = True)
tass.dropna(inplace = True)

In [ ]:
# Remove duplicates from washpost
cnbc.drop_duplicates(inplace = True)

In [ ]:
nytimes.drop_duplicates(inplace = True)

In [ ]:
bbc.drop_duplicates(inplace = True)

In [ ]:
ukrin.drop_duplicates(inplace = True)

In [ ]:
cna.drop_duplicates(inplace = True)

In [ ]:
tass.drop_duplicates(inplace = True)

# Information Extraction

In [ ]:
# Import necessary libraries
import spacy
import neuralcoref

def coreference_resolution(data):
  # Load the model
  nlp = spacy.load("en_core_web_lg")
  neuralcoref.add_to_pipe(nlp)

  new_str = ""
  for index, row in data.iterrows():
    new_str = unicodedata.normalize("NFKD", str(row["body"]))
    doc = nlp(new_str)  
    row["body"] = doc._.coref_resolved

  return data

100%|██████████| 40155833/40155833 [00:02<00:00, 16105161.99B/s]


In [ ]:
def ner(data_file, ner_list):
  for index, row in data_file.iterrows():
    temp_ner_article = []
    new_str = unicodedata.normalize("NFKD", str(row["body"]))
    test = client.annotate(new_str)
    ignore_word = ['cnbc', 'CNBC', 'of', 'He','he','his','she','She','her']
    unique_list = []
    for i, sent in enumerate(test.sentence):
        for t in sent.mentions:
          if t.entityMentionText not in ignore_word:
            if t.ner == "PERSON" or t.ner == "ORGANIZATION" or t.ner == "COUNTRY":
              if t.entityMentionText not in unique_list:
                unique_list.append(t.entityMentionText)
                temp_ner_article.append([t.entityMentionText, t.ner])
    ner_list.append(temp_ner_article)

In [ ]:
!python -m spacy download en_core_web_lg 

     |████████████████████████████████| 826.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
# Return unique ne from news sources
def unique_ne(data):
  unique_ne = []
  for article in data:
    for ne in article:
      if ne[0] not in unique_ne:
        unique_ne.append(ne[0])
  return unique_ne

In [ ]:
cnbc = coreference_resolution(cnbc)

In [ ]:
nytimes = coreference_resolution(nytimes)

In [ ]:
bbc = coreference_resolution(bbc)

In [ ]:
ukrin = coreference_resolution(ukrin)

In [ ]:
cna = coreference_resolution(cna)

In [ ]:
tass = coreference_resolution(tass)

In [ ]:
cnbc_ner = []
ner(cnbc, cnbc_ner)

In [ ]:
cnbc_unique_ne = unique_ne(cnbc_ner)

In [ ]:
cnbc_ner[0]

In [ ]:
cnbc_unique_ne

In [ ]:
nytimes_ner = []
ner(nytimes, nytimes_ner)

In [ ]:
nytimes_unique_ne = unique_ne(nytimes_ner)

In [ ]:
nytimes_ner[0]

In [ ]:
bbc_ner = []
ner(bbc, bbc_ner)

In [ ]:
bbc_unique_ne = unique_ne(bbc_ner)

In [ ]:
bbc_ner[2]

In [ ]:
ukrin_ner = []
ner(ukrin, ukrin_ner)

In [ ]:
ukrin_unique_ne = unique_ne(ukrin_ner)

In [ ]:
ukrin_ner[0]

In [ ]:
cna_ner = []
ner(cna, cna_ner)

In [ ]:
cna_unique_ne = unique_ne(cna_ner)

In [ ]:
cna_ner[0]

In [ ]:
tass_ner = []
ner(tass, tass_ner)

In [ ]:
tass_unique_ne = unique_ne(tass_ner)

In [ ]:
tass_ner[0]

# Sentence Splitting

In [ ]:
def sentence_split(data_file, article_list):
    for index, row in data_file.iterrows():
        sentence = []
        new_str = unicodedata.normalize("NFKD", str(row["body"]))
        text = new_str.split(". ")
        article_list.append(text)

In [ ]:
cnbc_articles = []
sentence_split(cnbc, cnbc_articles)

In [ ]:
cnbc_articles[0]

In [ ]:
nytimes_articles = []
sentence_split(nytimes, nytimes_articles)

In [ ]:
bbc_articles = []
sentence_split(bbc, bbc_articles)

In [ ]:
ukrin_articles = []
sentence_split(ukrin, ukrin_articles)

In [ ]:
cna_articles = []
sentence_split(cna, cna_articles)

In [ ]:
tass_articles = []
sentence_split(tass, tass_articles)

In [ ]:
tass_articles

# Sentiments analysis

In [ ]:
# Return polarity with positve, negative, neutral
def sentiments(row):
    if row['Polarity'] > 0:
        return "Positive"
    elif row['Polarity'] == 0:
        return "Neutral"
    else:
        return "Negative"

In [ ]:
# sentence that NE exist
def entity_sentence(data, ner_list, article_entity):
  for i in range(len(data)):
    entity_sen_list = []
    for sentence in data[i]:
      for ner in ner_list[i]:
        if ner[0] in sentence:
          entity_sen_list.append(sentence)
          break
          
    article_entity.append(entity_sen_list)

In [ ]:
# get all news article headline
def get_title(data):
  title_list = []
  for index, row in data.iterrows():
    title_list.append(row["title"])
  
  return title_list


In [ ]:
# Group article by entity
def group_article_by_entity(ner_list, entity_sentence, group_entity_article): 
  for i in range(len(ner_list)):
    # print(cnbc_ner[i][0])
    ner_group = []
    for j in range(len(ner_list[i])):
      word = ner_list[i][j][0]
      output = []
      for sentence in entity_sentence[i]:
        if word in sentence:
          output.append(sentence)
          # print(output)
      ner_group.append([word, output])
    group_entity_article.append(ner_group)
    
  

  for i in group_entity_article:
    for j in i:
      j.append(len(j[1]))
      j[1] = " ".join(str(sen) for sen in j[1])

In [ ]:
# Get sentimetns by entity from each article
def get_sentiments_by_entity_for_each_article(news_sources, group_entity_article, sentiments_list):
  title = get_title(news_sources)
  for i in range(len(group_entity_article)):
      for j in group_entity_article[i]:
        token = tokenizer.tokenize(j[1])
        a = 0
        b = 0
        for s in token:
          txt= TextBlob(s)
          a += txt.sentiment.polarity
          b += txt.sentiment.subjectivity
        # title, polarity, subjectivity, entity, no of sentence
        sentiments_list.append([title[i], a, b, j[0], j[2]])


In [ ]:
# # Get unique entity sentiments from all articles [simple average]
# def unique_entity_sentiemnt(unique_ne_list, sentiments_list, entity_sentiment_list):
#   for i in unique_ne_list:
#     subj = 0
#     polarity = 0
#     count = 0
#     for j in sentiments_list:
#       if j[3] == i:
#         count += 1
#         subj += j[2]
#         polarity += j[1]
#         # print(j)

#     avg_subj = subj/count
#     avg_polarity = polarity/count
#     entity_sentiment_list.append([i, avg_polarity, avg_subj])

#     print(avg_polarity)


In [ ]:
# Get unique entity sentiments from all articles [weighted average]
def unique_entity_sentiemnt(unique_ne_list, sentiments_list, entity_sentiment_list):
  for i in unique_ne_list:
    subj = 0
    polarity = 0
    total_sen = 0
    temp_list = []
    avg_polarity = 0
    avg_subj = 0
    for j in sentiments_list:
      if j[3] == i:
        total_sen += j[4]
        temp_list.append([j[1], j[2], j[4]])
    # [polarity, subjectivity, total sen]
    for item in temp_list:
      if total_sen != 0:
        avg_polarity += item[0] * (item[2]/total_sen)
        avg_subj += item[1] * (item[2]/total_sen)
    entity_sentiment_list.append([i, avg_polarity, avg_subj])

In [ ]:
#Remove Stopwords from article 
def remove_stopword(articles):
  stopwords = nltk.corpus.stopwords.words('english')
  for i in articles:
    text = i[1].lower()
    words = re.sub(r'[^\w\s]', '', text).split()
    i[1] = ' '.join([str(word) for word in words if word not in stopwords])

In [ ]:
remove_stopword(cnbc_articles)

In [ ]:
remove_stopword(bbc_articles)

In [ ]:
remove_stopword(nytimes_articles)

In [ ]:
remove_stopword(ukrin_articles)

In [ ]:
remove_stopword(cna_articles)

In [ ]:
remove_stopword(tass_articles)

# CNBC

In [ ]:
cnbc_entity_sen = []
entity_sentence(cnbc_articles, cnbc_ner, cnbc_entity_sen)

In [ ]:
cnbc_entity_sen

In [ ]:
cnbc_ner

In [ ]:
cnbc_sentiment=[]
# [Entity, article sen, No of sen]
group_cnbc = []

In [ ]:
group_article_by_entity(cnbc_ner, cnbc_entity_sen, group_cnbc)

In [ ]:
get_sentiments_by_entity_for_each_article(cnbc, group_cnbc, cnbc_sentiment)

In [ ]:
cnbc_entity_sentiment = []
unique_entity_sentiemnt(cnbc_unique_ne, cnbc_sentiment, cnbc_entity_sentiment)

In [ ]:
cnbc_entity_sentiment

In [ ]:
df_textblob = pd.DataFrame(cnbc_entity_sentiment, columns =['Entity', 'Polarity', 'Subjectivity']) 
df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

In [ ]:
pd.set_option('display.max_rows', None)
df_textblob

In [ ]:
df_textblob.to_excel("cnbc.xlsx", index=False)

In [ ]:
# df_textblob = pd.DataFrame(cnbc_sentiment, columns =['Title', 'Polarity', 'Subjectivity', 'NE']) 
# df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

# Nytimes

In [ ]:
ny_sentiment=[]
ny_entity_sen = []
group_ny = []
entity_sentence(nytimes_articles, nytimes_ner, ny_entity_sen)

In [ ]:
group_article_by_entity(nytimes_ner, ny_entity_sen, group_ny)

In [ ]:
get_sentiments_by_entity_for_each_article(nytimes, group_ny, ny_sentiment)

In [ ]:
nytimes_entity_sentiment = []
unique_entity_sentiemnt(nytimes_unique_ne, ny_sentiment, nytimes_entity_sentiment)

In [ ]:
df_textblob = pd.DataFrame(nytimes_entity_sentiment, columns =['Named Entity', 'Polarity', 'Subjectivity']) 
df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

In [ ]:
pd.set_option('display.max_rows', None)
df_textblob

In [ ]:
df_textblob.to_excel("nytimes.xlsx", index=False)

# BBC

In [ ]:
bbc_sentiment=[]
bbc_entity_sen = []
group_bbc = []
entity_sentence(bbc_articles, bbc_ner, bbc_entity_sen)

In [ ]:
group_article_by_entity(bbc_ner, bbc_entity_sen, group_bbc)

In [ ]:
get_sentiments_by_entity_for_each_article(bbc, group_bbc, bbc_sentiment)

In [ ]:
bbc_entity_sentiment = []
unique_entity_sentiemnt(bbc_unique_ne, bbc_sentiment, bbc_entity_sentiment)

In [ ]:
df_textblob = pd.DataFrame(bbc_entity_sentiment, columns =['Named Entity', 'Polarity', 'Subjectivity']) 
df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

In [ ]:
# pd.set_option('display.max_rows', None)
df_textblob

In [ ]:
df_textblob.to_excel("bbc.xlsx", index=False)

# ukrinform

In [ ]:
ukrin_sentiment=[]
ukrin_entity_sen = []
group_ukrin = []
entity_sentence(ukrin_articles, ukrin_ner, ukrin_entity_sen)

In [ ]:
group_article_by_entity(ukrin_ner, ukrin_entity_sen, group_ukrin)

In [ ]:
get_sentiments_by_entity_for_each_article(ukrin, group_ukrin, ukrin_sentiment)

In [ ]:
ukrin_entity_sentiment = []
unique_entity_sentiemnt(ukrin_unique_ne, ukrin_sentiment, ukrin_entity_sentiment)

In [ ]:
df_textblob = pd.DataFrame(ukrin_entity_sentiment, columns =['Named Entity', 'Polarity', 'Subjectivity']) 
df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

In [ ]:
# pd.set_option('display.max_rows', None)
df_textblob

In [ ]:
df_textblob.to_excel("ukrinform.xlsx", index=False)

# CNA

In [ ]:
cna_sentiment=[]
cna_entity_sen = []
group_cna = []
entity_sentence(cna_articles, cna_ner, cna_entity_sen)

In [ ]:
group_article_by_entity(cna_ner, cna_entity_sen, group_cna)

In [ ]:
get_sentiments_by_entity_for_each_article(cna, group_cna, cna_sentiment)

In [ ]:
cna_entity_sentiment = []
unique_entity_sentiemnt(cna_unique_ne, cna_sentiment, cna_entity_sentiment)

In [ ]:
df_textblob = pd.DataFrame(cna_entity_sentiment, columns =['Named Entity', 'Polarity', 'Subjectivity']) 
df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

In [ ]:
pd.set_option('display.max_rows', False)
df_textblob

,Named Entity,Polarity,Subjectivity,Sentiments
0,Fahda Fahd,0.056944,0.361271,Positive
1,Kia,0.100000,0.345833,Positive
2,AFP,0.244702,0.621190,Positive
3,Saudi Arabia,0.200000,0.200000,Positive
4,Insaf,0.000000,0.500000,Neutral
5,Aya Diab,0.300000,0.566667,Positive
6,Delta,0.000000,0.000000,Neutral
7,Huawei,-0.085185,0.212963,Negative
8,Tencent,-0.085185,0.212963,Negative
...,...,...,...,...


In [ ]:
df_textblob.to_excel("cna.xlsx", index=False)

# Tass

In [ ]:
tass_sentiment=[]
tass_entity_sen = []
group_tass = []
entity_sentence(tass_articles, tass_ner, tass_entity_sen)

In [ ]:
group_article_by_entity(tass_ner, tass_entity_sen, group_tass)

In [ ]:
get_sentiments_by_entity_for_each_article(tass, group_tass, tass_sentiment)

In [ ]:
tass_sentiment

In [ ]:
tass_entity_sentiment = []
unique_entity_sentiemnt(tass_unique_ne, tass_sentiment, tass_entity_sentiment)

In [ ]:
df_textblob = pd.DataFrame(tass_entity_sentiment, columns =['Named Entity', 'Polarity', 'Subjectivity']) 
df_textblob["Sentiments"] = df_textblob.apply(sentiments, axis= 1)

In [ ]:
df_textblob

,Named Entity,Polarity,Subjectivity,Sentiments
0,TASS,0.218083,0.787963,Positive
1,Ukraine,0.227138,1.488419,Positive
2,KIEV,0.032143,0.578205,Positive
3,Vladimir Zelensky,0.037164,1.050835,Positive
4,EU,0.241126,1.177887,Positive
5,Andrey Yermak,0.000000,0.300000,Neutral
6,France,0.124652,0.552534,Positive
7,Mark Rutte,0.016667,0.366667,Positive
8,Russia,0.218330,1.904049,Positive
...,...,...,...,...


In [ ]:
df_textblob.to_excel("tass.xlsx", index=False)

# Stop stanza

In [ ]:
# Shut down the background CoreNLP server
client.stop()

time.sleep(10)
!ps -o pid,cmd | grep java